In [1]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf, statsmodels.api as sm
import datetime
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.float_format', '{:,.4f}'.format)

<p style="font-size:2em; font-weight:400; margin:.2em 0;">Introduction</p>

The objective of this Notebook is to demonstrate a concept to solve a case similar to the EDUVISTA case in the main exercise in this repo. That is, we want to estimate the effects of interventions on the event rate. However, we relax a strong key assumption of the previous exercise. In this notebook, intervention effects decay at an unknown rate $\rho$, i.e. we assume a geometric decay. The parameter space for $\rho$ includes $1$ so the model nests a world with no decay.

The idea is to employ a state space stock formulation where the latent effect stock $s$ for cross section $i$ at time $t$ evolves recursively

$$s_{i,t} = \rho s_{i,t-1} + x_{i,t}, $$

where $x$ is an intervention. The latent stock replaces the cumulative sum of interventions in the familiar Poisson regression problem with a cross section fixed effect $\alpha_i$:

$$log\ \lambda_{i,t} = \alpha_i + \beta s_{i,t}$$

We wish to estimate $\rho$, $\alpha_i$, and $\beta$, the coefficient of interest that captures intervention efficiency. This is achieved by a profile likelihood nested MLE approach. For each candidate $\rho$, we recompute the state via the recursion and fit a Poisson GLM that maximizes the log likelihood over the regression coefficients conditional on that value of $\rho$. The value that maximizes the profile log likelihood is our estimate of $\rho$. The regression coefficients are then taken at the estimated decay parameter. To obtain robust standard errors for all parameters, we subsequently use a bootstrap approach.

We still rely on the assumption of immediate intervention effects. However, the method can be extended to allow for an onset delay. An alternative to the state space approach is a distributed lag model which contains lags of interventions and estimates their coefficients. That is fairly flexible with no assumptions on the shape of the decay but comes at the cost of potentially many parameters and multicollinearity across lags.

In the first section, we generate data that replicate the essential case of the main exercise with the exception of decaying interventions. For convenience we use one intervention. The baseline model uses the true unobserved decay parameter and we estimate only the intervention effect in order to obtain a benchmark for the profile likelihood estimates. In the last section, we employ a bootstrap approach to derive robust standard errors for all parameters.

# Data Generation

Let us generate some data for the interventions and events. 

In [2]:
np.random.seed(42)

In [3]:
T = 200 # Time periods
n = 100 # Cross sections

In [4]:
# Interventions
lambda_day = 2 / T # Roughly 7 interventions per year
interventions = np.zeros((n, T), dtype=int)

In [5]:
for i in range(n):
    int_per_school = np.random.poisson(lam=lambda_day * T)
    int_per_school = min(int_per_school, T)
    days = np.random.choice(T, size=int_per_school, replace=False)
    interventions[i, days] = 1

In [6]:
# Intervention decay
rho_true = .9

In [7]:
baseline_rate = np.log(0.04) # baseline rate
beta_true = np.log(2) # double the effect after the innovation
school_shift = np.random.uniform(-0.015, 0.015, size=n) # school speific rate

In [8]:
def state_value_fct(interventions, rho, s_initial=0):
    n, T = interventions.shape
    state_value = np.zeros((n, T))
    state = np.full(n, s_initial)
    interventions = interventions.astype(float)
    
    for t in range(T):
        state = rho * state + interventions[:, t]
        state_value[:, t] = state
    return state_value

In [9]:
# Simulate intervention state
s_sim = state_value_fct(interventions,rho_true)

In [10]:
# event rate
daily_lambda = np.exp(baseline_rate + school_shift[:, None] + beta_true * s_sim)

In [11]:
# Add some noise to the event rate
sigma_lambda = 0.06  
daily_lambda_tilde = daily_lambda * np.exp(np.random.normal(0, sigma_lambda, size=daily_lambda.shape))

In [12]:
# Generate events
y = np.random.poisson(lam=daily_lambda_tilde)

# Baseline Estimation (True Decay Parameter)

First, we fit a baseline Poisson model based on the true decay parameter which is, in reality, unknown.

In [13]:
school_ids = np.arange(n)
days = np.arange(T)

In [14]:
df = pd.DataFrame({
    "y": y.ravel(),
    "t": np.tile(np.arange(T), n),
    "school_id": np.repeat(school_ids, T),
    "intervention_state": s_sim.ravel()
    })

In [15]:
df = df.sort_values(["school_id", "t"], kind="mergesort").reset_index(drop=True)

In [16]:
df.y  = df.y.astype(int)

In [17]:
df.head(2)

,y,t,school_id,intervention_state
0,0,0,0,0.0000
1,0,1,0,0.0000


In [18]:
formula_base = 'y ~ intervention_state + C(school_id)'

In [19]:
model = smf.glm(formula=formula_base,data=df,family=sm.families.Poisson()).fit(cov_type='cluster', cov_kwds={'groups': df['school_id']})

In [20]:
mask = ~model.params.index.str.contains(r"C\(school_id\)")

In [21]:
idx = model.params[mask].index.intersection(model.conf_int().index)

In [22]:
coefs = model.params.loc[idx]
pvals = model.pvalues.loc[idx]
conf = model.conf_int().loc[idx]
irr     = np.exp(coefs)
irr_ci  = np.exp(conf)

In [23]:
df_baseline = pd.DataFrame({
    "coef": coefs,
    "pval": pvals,
    "ci_lower": conf[0],
    "ci_upper": conf[1],
    "IRR": irr,
    "IRR_ci_lower": irr_ci[0],
    "IRR_ci_upper": irr_ci[1],
})

In [24]:
df_baseline

,coef,pval,ci_lower,ci_upper,IRR,IRR_ci_lower,IRR_ci_upper
Intercept,-2.5731,0.0000,-2.6257,-2.5204,0.0763,0.0724,0.0804
intervention_state,0.6221,0.0000,0.3547,0.8895,1.8629,1.4258,2.4339


**Interpretation**

Intercept: baseline expected count per day

In [25]:
np.round(np.exp(model.params.iloc[0]),3)

0.076

Intervention_state: a one-unit increase in the state (an additional intervention) multiplies the expected count in that period by the IRR (instantaneous effect). For a one-off intervention, the effect on later periods decays geometrically at rate $\rho$. The half-life (periods until the effect halves) is:

In [26]:
if rho_true < 1:
    half_life = np.round(np.log(0.5) / np.log(rho_true), 2)
    print("Half-life:", half_life)
else:
    print('Interventions do not decay and thus, have no half-life')

Half-life: 6.58


# Profile-likelihood MLE for Intervention Effects and Decay

In the previous section I estimated intervention effects assuming a known decay parameter $\rho$. In practice the parameter is unknown, so I use a profile-likelihood MLE as described above. 

In [27]:
def poisson_reg_joint(p,df,interventions, formula):
    rho = float(p[0])     
    state_values = state_value_fct(interventions, rho)
    df_tmp = df.copy()
    df_tmp['intervention_state'] = state_values.ravel()  
    fit = smf.glm(formula=formula, data=df_tmp, family=sm.families.Poisson()).fit() # Notice that there are no robust SE in this specification. 
    return -fit.llf # negative likelihood value

In [28]:
rho_initial = 0.7
initial_val = np.array([rho_initial]) 

In [29]:
opt = minimize(poisson_reg_joint, x0 = initial_val, args=(df, interventions,formula_base),  bounds=[(0.2, 1)], method="L-BFGS-B") 

In [30]:
rho_hat = float(opt.x[0])

In [31]:
rho_hat

0.7921897413275177

**Compute state values and estimated intervention coefficients based on the estimated decay coefficient (rho)**

In [32]:
s_sim_hat = state_value_fct(interventions, rho_hat, 0) # the state based on the the estimated rho
df_hat = df.copy()
df_hat["intervention_state"] = s_sim_hat.ravel()

In [33]:
fit_joint = smf.glm(formula=formula_base, data=df_hat,
             family=sm.families.Poisson()).fit()

In [34]:
mask = ~fit_joint.params.index.str.contains(r"C\(school_id\)")

In [35]:
idx = fit_joint.params[mask].index.intersection(fit_joint.conf_int().index)

In [36]:
coefs = fit_joint.params.loc[idx]
pvals = fit_joint.pvalues.loc[idx]
conf = fit_joint.conf_int().loc[idx]
irr     = np.exp(coefs)
irr_ci  = np.exp(conf)

In [37]:
df_joint = pd.DataFrame({
    "coef": coefs,
    "pval": pvals,
    "ci_lower": conf[0],
    "ci_upper": conf[1],
    "IRR": irr,
    "IRR_ci_lower": irr_ci[0],
    "IRR_ci_upper": irr_ci[1],
})

In [38]:
np.round(rho_hat,3), rho_true

(0.792, 0.9)

In [39]:
df_baseline[['coef','pval']]

,coef,pval
Intercept,-2.5731,0.0000
intervention_state,0.6221,0.0000


In [40]:
df_joint[['coef','pval']]

,coef,pval
Intercept,-2.5479,0.0000
intervention_state,0.8505,0.0000


Let us focus on the comparison of both tables. We expect a larger interventions coefficient if the estimated value for $\rho$ is smaller than the true value because a lower $\rho$ makes the constructed state less persistent (smaller on average), so the model compensates by increasing the coefficient on the state; conversely, overestimating $\rho$ leads to a smaller intervention coefficient.

The usual standard errors for the intercept and the state/intervention coefficient from this procedure are not reliable: they are conditional on $\hat{\rho}$ and ignore its estimation uncertainty. This can be addressed by estimating the full joint likelihood or by using bootstrap SEs. I use a cluster bootstrap rather than joint-likelihood SEs because the bootstrap relies on empirical resampling rather than analytic, likelihood-based approximations to the sampling covariance, and it naturally propagates the uncertainty in $\rho$ . 

# Bootstrapped SE

In [41]:
B = 100 # Number of boostrap samples
ids = df.school_id.unique()
N = len(ids)

In [42]:
boot_stats = []
formula_boot = 'y ~ intervention_state + C(school_id, Treatment(reference=0))'

for b in range(B):
    if (b + 1) % 100 == 0:
        current_time = datetime.datetime.now().strftime("%H:%M:%S")
        print(f"Iteration {b + 1} at time: {current_time}")
        
    # Sample with replacement and make sure school_id == 0 is always the baseline (intercept)
    remaining_schools = np.random.choice(ids[ids != 0], size=N - 1, replace=True)
    sampled_ids = np.concatenate(([0], remaining_schools))

    df_b = pd.concat([df[df.school_id == i] for i in sampled_ids], ignore_index=True) # Bootstrap panel with repetitive IDs
    interventions_b = np.concatenate([interventions[ids == i, :] for i in sampled_ids],axis=0) # Get the respective intervention rows

    # Optimize rho
    res = minimize(poisson_reg_joint, x0=rho_initial,
                   args=(df_b, interventions_b,formula_boot),
                   bounds=[(0.2, 1.0)], method="L-BFGS-B")

    rho_b = res.x[0]
    s_b = state_value_fct(interventions_b, rho_b)
    df_fit = df_b.copy()
    df_fit["intervention_state"] = s_b.ravel()
    
    # Refit GLM at rho_b to get betas on this bootstrap sample
    fit_b = smf.glm(formula=formula_boot,
                data=df_fit,
                family=sm.families.Poisson()
               ).fit()
    
    intercept_b = fit_b.params["Intercept"]
    beta_b = fit_b.params["intervention_state"]
    
    boot_stats.append({"rho":rho_b, "intercept": intercept_b, "beta": beta_b})

Iteration 100 at time: 13:57:28


In [43]:
boot = pd.DataFrame(boot_stats)

In [44]:
point = pd.Series({"rho": rho_hat, "intercept": df_joint.loc['Intercept','coef'], "beta": df_joint.loc['intervention_state','coef']},name="point_estimate")

In [45]:
conf = .95
alpha = 1 - conf

In [46]:
bootstrap_summary = pd.DataFrame({
    "ci_lo":  boot.quantile(alpha/2),
    "ci_hi":  boot.quantile(1 - alpha/2),
    "mean_bs":   boot.mean(),
    "median_bs": boot.median(),
    "se_bs":     boot.std(ddof=1),
}).loc[["rho","intercept","beta"]] 

In [47]:
summary = pd.concat([point, bootstrap_summary.reindex(point.index)], axis=1)

In [48]:
summary

,point_estimate,ci_lo,ci_hi,mean_bs,median_bs,se_bs
rho,0.7922,0.6147,0.8999,0.7935,0.8127,0.0786
intercept,-2.5479,-2.6037,-2.5062,-2.5512,-2.5495,0.0259
beta,0.8505,0.5703,1.1444,0.8127,0.8122,0.1574


The table reports point estimates from the profile-likelihood (nested MLE) and 95% bootstrap percentile CIs, along with bootstrap mean, median, and SE.